In [1]:
import boto3
import json
import psycopg2
import base64
from faker import Faker
import uuid
from datetime import datetime, timedelta
import random
import csv
import io

## Set Parameters

In [2]:
env='dev'
region_name = 'us-east-1'
account_id=183286346090
secret_name = f'ticketing-postgres-credentials-{env}'
bucket_name = f'ticketing-data-{account_id}-{env}'

## Get Postgres DB secret

In [3]:
def get_secret(secret_name):
    """Retrieves the secret from Secrets Manager."""
    
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except Exception as e:
        if e.response['Error']['Code'] == 'DecryptionFailureException':
            # The secret can't be decrypted using the provided KMS key.
            raise e
        elif e.response['Error']['Code'] == 'InternalServiceErrorException':
            # An error occurred inside Secrets Manager.
            raise e
        elif e.response['Error']['Code'] == 'InvalidParameterException':
            # You provided an invalid value for a parameter.
            raise e
        elif e.response['Error']['Code'] == 'InvalidRequestException':
            # You provided a bad request.
            raise e
        elif e.response['Error']['Code'] == 'ResourceNotFoundException':
            # The requested secret was not found.
            raise e
    else:
        # Decrypted secrets are available directly.
        if 'SecretString' in get_secret_value_response:
            secret = get_secret_value_response['SecretString']
            return json.loads(secret)  # Return secret as a dictionary
        else:
            decoded_binary_secret = base64.b64decode(get_secret_value_response['SecretBinary'])
            return decoded_binary_secret

In [4]:
try:
    secret = get_secret(secret_name)
    db_host = secret["host"]
    db_name = secret["dbname"]
    db_user = secret["username"]
    db_password = secret["password"]    

except Exception as e:
    print(f"Error retrieving secret or connecting to database: {e}")

## Create Tables

In [5]:
def create_tables(conn):
    """Creates the tables in the PostgreSQL database."""

    cursor = conn.cursor()

    tables = {
        "venues": """
            CREATE TABLE venues (
                venue_id UUID PRIMARY KEY DEFAULT gen_random_uuid(),
                venue_name VARCHAR(255) NOT NULL,
                location VARCHAR(255)
            )
        """,
        "organizers": """
            CREATE TABLE organizers (
                organizer_id UUID PRIMARY KEY DEFAULT gen_random_uuid(),
                organizer_name VARCHAR(255) NOT NULL
            )
        """,
        "resellers": """
            CREATE TABLE resellers (
                reseller_id UUID PRIMARY KEY DEFAULT gen_random_uuid(),
                reseller_name VARCHAR(255) NOT NULL,
                location VARCHAR(255)
            )
        """,
        "customers": """
            CREATE TABLE customers (
                customer_id UUID PRIMARY KEY DEFAULT gen_random_uuid(),
                customer_name VARCHAR(255) NOT NULL
            )
        """,
        "sales_channels": """
            CREATE TABLE sales_channels (
                sales_channel_id UUID PRIMARY KEY DEFAULT gen_random_uuid(),
                channel_name VARCHAR(255) NOT NULL
            )
        """,
        "events": """
            CREATE TABLE events (
                event_id UUID PRIMARY KEY DEFAULT gen_random_uuid(),
                event_name VARCHAR(255) NOT NULL,
                event_type VARCHAR(100) NOT NULL,  -- Category of event (e.g., Concert, Sports, Theater)
                venue_id UUID REFERENCES venues(venue_id) NOT NULL
            )
        """,
        "sales": """
            CREATE TABLE sales (
                sale_id UUID PRIMARY KEY DEFAULT gen_random_uuid(),
                event_id UUID REFERENCES events(event_id) NOT NULL, -- Added event_id reference
                venue_id UUID REFERENCES venues(venue_id) NOT NULL,
                organizer_id UUID REFERENCES organizers(organizer_id) NOT NULL,
                reseller_id UUID REFERENCES resellers(reseller_id),
                customer_id UUID REFERENCES customers(customer_id) NOT NULL,
                sales_channel_id UUID REFERENCES sales_channels(sales_channel_id) NOT NULL,
                ticket_quantity INT NOT NULL,
                total_amount DECIMAL(10, 2) NOT NULL,
                sale_date TIMESTAMP NOT NULL
            )
        """,
        "partnerships": """
            CREATE TABLE partnerships (
                partnership_id UUID PRIMARY KEY DEFAULT gen_random_uuid(),
                organizer_id UUID REFERENCES organizers(organizer_id) NOT NULL,
                reseller_id UUID REFERENCES resellers(reseller_id) NOT NULL,
                commission_rate DECIMAL(5, 2) NOT NULL,
                start_date DATE,
                end_date DATE
            )
        """
    }

    for table_name, create_statement in tables.items():
        try:
            cursor.execute(create_statement)
            conn.commit()
            print(f"Table '{table_name}' created successfully.")
        except psycopg2.errors.DuplicateTable:
            print(f"Table '{table_name}' already exists.")
            conn.rollback()  # Rollback transaction in case of error
        except Exception as e:
            print(f"Error creating table '{table_name}': {e}")
            conn.rollback()

    cursor.close()

In [ ]:
try:
    # Use the retrieved credentials to connect to PostgreSQL
    conn = psycopg2.connect(host=db_host, database=db_name, user=db_user, password=db_password)
    create_tables(conn)
except Exception as e:
    print(f"Error connecting to database: {e}")
finally:
    if conn:
        conn.close()

## Populate test database with fake data

In [10]:
import uuid
from faker import Faker
from datetime import datetime, timedelta
from psycopg2.extras import register_uuid

# Register the UUID extension
register_uuid()

fake = Faker()

# Define a list of possible event types
EVENT_TYPES = ["Concert", "Sports", "Theater", "Conference", "Festival", "Exhibition"]

def populate_tables(conn, num_venues, num_organizers, num_resellers, num_customers, num_partnerships, num_events, num_sales):
    """Populates tables with fake data."""
    cursor = conn.cursor()

    try:
        # Venues
        venues = []
        for _ in range(num_venues):
            venue_id = uuid.uuid4()
            cursor.execute("INSERT INTO venues (venue_id, venue_name, location) VALUES (%s, %s, %s)", 
                           (venue_id, fake.company(), fake.city()))
            venues.append(venue_id)

        # Organizers
        organizers = []
        for _ in range(num_organizers):
            organizer_id = uuid.uuid4()
            cursor.execute("INSERT INTO organizers (organizer_id, organizer_name) VALUES (%s, %s)", 
                           (organizer_id, fake.company()))
            organizers.append(organizer_id)

        # Resellers
        resellers = []
        for _ in range(num_resellers):
            reseller_id = uuid.uuid4()
            cursor.execute("INSERT INTO resellers (reseller_id, reseller_name, location) VALUES (%s, %s, %s)", 
                           (reseller_id, fake.company(), fake.city()))
            resellers.append(reseller_id)

        # Customers
        customers = []
        for _ in range(num_customers):
            customer_id = uuid.uuid4()
            cursor.execute("INSERT INTO customers (customer_id, customer_name) VALUES (%s, %s)", 
                           (customer_id, fake.name()))
            customers.append(customer_id)

        # Partnerships
        for _ in range(num_partnerships):
            partnership_id = uuid.uuid4()
            partnership_start_date = fake.date_between(start_date=datetime(2017, 1, 1), end_date=datetime(2018, 12, 31))
            cursor.execute(
                "INSERT INTO partnerships (partnership_id, organizer_id, reseller_id, commission_rate, start_date, end_date) VALUES (%s, %s, %s, %s, %s, %s)",
                (partnership_id, 
                 fake.random_element(organizers), 
                 fake.random_element(resellers), 
                 fake.pydecimal(2, 2, positive=True), 
                 partnership_start_date, 
                 partnership_start_date + timedelta(days=+random.randint(720, 1080)))
            )

        # Sales Channels (fixed values)
        sales_channels = []
        for channel_name in ["on-the-site", "web", "mobile app"]:
            sales_channel_id = uuid.uuid4()
            cursor.execute("INSERT INTO sales_channels (sales_channel_id, channel_name) VALUES (%s, %s)", 
                           (sales_channel_id, channel_name))
            sales_channels.append(sales_channel_id)

        # Events
        events = []
        for _ in range(num_events):
            event_id = uuid.uuid4()
            venue_id = fake.random_element(venues)  # Ensure event is linked to a valid venue
            event_type = fake.random_element(EVENT_TYPES)
            event_name = fake.catch_phrase()
            cursor.execute("INSERT INTO events (event_id, event_name, event_type, venue_id) VALUES (%s, %s, %s, %s)", 
                           (event_id, event_name, event_type, venue_id))
            events.append((event_id, venue_id, event_name))  # Store event with its venue for reference

        # Sales
        for _ in range(num_sales):
            sale_id = uuid.uuid4()
            
            # Choose a random event and ensure the venue is consistent
            event_id, venue_id, event_name = fake.random_element(events)
            
            cursor.execute(
                "INSERT INTO sales (sale_id, event_id, venue_id, organizer_id, reseller_id, customer_id, sales_channel_id, ticket_quantity, total_amount, sale_date) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)",
                (
                    sale_id,
                    event_id,  # Reference the correct event
                    venue_id,  # Ensure the venue matches the event
                    fake.random_element(organizers),
                    fake.random_element(resellers) if _ % 3 != 0 else None,
                    fake.random_element(customers),
                    fake.random_element(sales_channels),
                    fake.random_int(min=1, max=10),
                    fake.pydecimal(5, 2, positive=True),
                    fake.date_time_between(start_date=datetime(2019, 1, 1), end_date=datetime(2022, 12, 31))
                )
            )

        conn.commit()
        print("Tables populated successfully.")
        return resellers, events

    except Exception as e:
        conn.rollback()
        print(f"Error populating tables: {e}")

    cursor.close()
    return None

In [11]:
conn = psycopg2.connect(host=db_host, database=db_name, user=db_user, password=db_password)    

# Populate tables with fake data
num_venues = 10
num_events = 5
num_organizers = 5
num_resellers = 3
num_customers = 500
num_partnerships = 5
num_sales = 5000

resellers, events = populate_tables(conn, num_venues, num_organizers, num_resellers, num_customers, num_partnerships, num_events, num_sales)

conn.close()

Tables populated successfully.


## Populate CSV

In [12]:
import io
import csv
import random
import boto3
import uuid
from datetime import datetime, timedelta
from faker import Faker

fake = Faker()

def populate_csv_to_s3(bucket_name, resellers, events, num_files):
    """Populates a CSV file with fake ticketing data, including event_id, and uploads it to S3."""

    sales_channels = ["office", "web", "mobile app"]
    office_locations = ["New York", "London", "Tokyo", "Sydney", "San Francisco"]

    transaction_id_counter = 1  # Initialize a global transaction ID counter    

    try:
        for file_num in range(num_files):                
            # Generate a random date for each file
            start_date = datetime(2019, 1, 1)
            end_date = datetime(2020, 12, 31)
            time_between_dates = end_date - start_date
            days_between_dates = time_between_dates.days
            random_number_of_days = random.randrange(days_between_dates)
            random_date = start_date + timedelta(days=random_number_of_days)
            date_str = random_date.strftime("%m%d%Y")

            # Create filename for each file
            filename = f"DailySales_{date_str}_{random.choice(resellers)}.csv"

            # Generate a random number of rows for each file
            num_rows = random.randint(10, 300)

            # Create CSV data in memory using StringIO
            csv_data = io.StringIO()  
            fieldnames = [
                "Transaction ID", "Event Name", "Number of Purchased Tickets", "Total Amount",
                "Sales Channel", "Customer First Name", "Customer Last Name", "Office Location", "Created Date"
            ]
            writer = csv.DictWriter(csv_data, fieldnames=fieldnames)
            writer.writeheader()

            for _ in range(num_rows):
                transaction_id = transaction_id_counter
                transaction_id_counter += 1                
                event_name = random.choice(events)[2]                
                num_tickets = random.randint(1, 5)
                total_amount = round(random.uniform(20, 200), 2)
                sales_channel = random.choice(sales_channels)
                first_name = fake.first_name()
                last_name = fake.last_name()
                office_location = random.choice(office_locations) if sales_channel == "office" else ""
                created_date = random_date.strftime("%Y-%m-%d")

                writer.writerow({
                    "Transaction ID": transaction_id,
                    "Event Name": event_name,
                    "Number of Purchased Tickets": num_tickets,
                    "Total Amount": total_amount,
                    "Sales Channel": sales_channel,
                    "Customer First Name": first_name,
                    "Customer Last Name": last_name,
                    "Office Location": office_location,
                    "Created Date": created_date
                })

            # Upload to S3
            s3 = boto3.client('s3')
            csv_string = csv_data.getvalue()  
            s3.put_object(Bucket=bucket_name, Key="sales/"+filename, Body=csv_string.encode('utf-8'))

            print(f"Successfully uploaded {filename} to S3 bucket {bucket_name}")

    except Exception as e:
        print(f"Error: {e}")

num_files = 1000
populate_csv_to_s3(bucket_name, resellers, events, num_files)

Successfully uploaded DailySales_04242019_4650b924-f628-4c30-b969-ab1a5108de00.csv to S3 bucket ticketing-data-183286346090-dev
Successfully uploaded DailySales_09052019_76ef3a86-1d6b-48c1-bd5d-e62febbdc5f6.csv to S3 bucket ticketing-data-183286346090-dev
Successfully uploaded DailySales_01202019_4650b924-f628-4c30-b969-ab1a5108de00.csv to S3 bucket ticketing-data-183286346090-dev
Successfully uploaded DailySales_12302020_4650b924-f628-4c30-b969-ab1a5108de00.csv to S3 bucket ticketing-data-183286346090-dev
Successfully uploaded DailySales_03202020_4650b924-f628-4c30-b969-ab1a5108de00.csv to S3 bucket ticketing-data-183286346090-dev
Successfully uploaded DailySales_12212019_4650b924-f628-4c30-b969-ab1a5108de00.csv to S3 bucket ticketing-data-183286346090-dev
Successfully uploaded DailySales_02202019_4969f644-6082-4385-b2f3-1a87c54fb4ea.csv to S3 bucket ticketing-data-183286346090-dev
Successfully uploaded DailySales_05062020_76ef3a86-1d6b-48c1-bd5d-e62febbdc5f6.csv to S3 bucket ticketin